In [1]:
# Install the project dependencies 
%pip install --quiet -U langchain_openai langchain_core langgraph
%pip install -qU langchain-deepseek-official
!pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Set up Env

import os, getpass 
from dotenv import load_dotenv
load_dotenv()


def _set_env(var:str) :
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}")
_set_env("OPENAI_API_KEY")

In [4]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "pr-weary-bull-48"

In [8]:
from langchain_openai import ChatOpenAI 


def multiply(a:int, b:int) -> int :
    """Multiply a and b 
    Args:
        a: first int
        b: second int
    """
    return a * b 

def sub (a:int, b:int) -> int:
    """ Substraction a and b 
    Args:
        a: first int 
        b: second int 
    """
    return a - b 

tools = [sub, multiply]

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)
    

In [10]:
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")


def assistant(state:MessagesState) :
    return {"messages": [llm_with_tools.invoke([sys_msg]+ state["messages"])]}


In [18]:

from langgraph.graph import START, StateGraph 
from langgraph.prebuilt import tools_condition, ToolNode 
from IPython.display import Image, display


builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant )
builder.add_node("tools", ToolNode(tools))


builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition
)

builder.add_edge("tools", "assistant")
# react_graph = builder.compile()


# Memeory adding into it 

In [19]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
react_graph_memory = builder.compile(checkpointer=memory)

In [25]:
config = {"configurable":{"thread_id": "1"}}

msgs = [HumanMessage(content="multiple 3 and 4")]
msgs = react_graph_memory.invoke({"messages": msgs}, config)
for m in msgs["messages"]:
    m.pretty_print()

================================ Human Message =================================

multiple 3 and 4
================================== Ai Message ==================================
Tool Calls:
  multiply (call_kbdQlUfFZ3gUlAnEnWRFGCd8)
 Call ID: call_kbdQlUfFZ3gUlAnEnWRFGCd8
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: multiply

12
================================== Ai Message ==================================

The product of 3 and 4 is 12.


In [26]:
msgs = HumanMessage(content="substract that with 4")

msgs = react_graph_memory.invoke({"messages": msgs}, config)
for m in msgs["messages"]:
    m.pretty_print()


================================ Human Message =================================

multiple 3 and 4
================================== Ai Message ==================================
Tool Calls:
  multiply (call_kbdQlUfFZ3gUlAnEnWRFGCd8)
 Call ID: call_kbdQlUfFZ3gUlAnEnWRFGCd8
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: multiply

12
================================== Ai Message ==================================

The product of 3 and 4 is 12.
================================ Human Message =================================

substract that with 4
================================== Ai Message ==================================
Tool Calls:
  sub (call_oueSGSS5LvSqd6onn3qX1Jij)
 Call ID: call_oueSGSS5LvSqd6onn3qX1Jij
  Args:
    a: 12
    b: 4
================================= Tool Message =================================
Name: sub

8
================================== Ai Message ==================================

Subtracti